## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
import tensorflow
import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser
print("Inports Inported")

Inports Inported


In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower',
                       interpolation='none')
print("Plot Data planted")

Plot Data planted


#### Setup hparams

In [3]:
hparams = create_hparams()
hparams.sampling_rate = 22050
print("Hparams Configurated")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Hparams Configurated


#### Load model from checkpoint

In [4]:
checkpoint_path = "pretrained/checkpoint_2000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()
print("Tacotron model loaded")

Tacotron model loaded


#### Load WaveGlow for mel2audio synthesis and denoiser

In [5]:
#waveglow_path = 'pretrained/waveglow_256channels_universal_v5.pt'
#waveglow = torch.load(waveglow_path)['model']
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)
print("Waveglow model loaded")

Using cache found in C:\Users\krzys/.cache\torch\hub\nvidia_DeepLearningExamples_torchhub


Waveglow model loaded


#### Prepare text input

In [6]:
text = "I am very sleepy!"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()
print("Text read")

Text read


#### Decode text input and plot results

In [7]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
print("Mells plotted")
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))


Warning! Reached max decoder steps
Mells plotted


#### Synthesize audio from spectrogram using WaveGlow

In [8]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
print("AudioFile Created")
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)


AudioFile Created


#### (Optional) Remove WaveGlow bias

In [9]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
print("Audio Created")
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate)


Audio Created
